In [39]:
import numpy as np
import pandas as pd
import dateutil
from pandasql import sqldf
import time

In [3]:
data = pd.read_csv(r'C:\Users\Masha\virtualenvs\tensorflow\data\phone_data.csv', sep=",")
data.head()

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile


In [15]:
# Запрос на вывод количества звонков, смс и данных, переданных за каждый месяц с помощью pandas
data.groupby(['month', 'item'])['date'].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

In [42]:
# Запрос на вывод количества звонков, смс и данных, переданных за каждый месяц с помощью pandasql
pysqldf = lambda q: sqldf(q, globals())
print (pysqldf("SELECT month, item, count(date) as date FROM data GROUP BY month, item;"))

      month  item  date
0   2014-11  call   107
1   2014-11  data    29
2   2014-11   sms    94
3   2014-12  call    79
4   2014-12  data    30
5   2014-12   sms    48
6   2015-01  call    88
7   2015-01  data    31
8   2015-01   sms    86
9   2015-02  call    67
10  2015-02  data    31
11  2015-02   sms    39
12  2015-03  call    47
13  2015-03  data    29
14  2015-03   sms    25


In [41]:
# Измерим время выполнения запроса на группировку набора данных с использованием функций агрегирования
# с помощью pandas и pandasql
class Profiler(object):
    def __enter__(self):
        self._startTime = time.time()
         
    def __exit__(self, type, value, traceback):
        print ("Elapsed time: {:.3f} sec".format(time.time() - self._startTime))
        
with Profiler() as p:
    data.groupby(['month', 'item'])['date'].count()
with Profiler() as p:
    pysqldf("SELECT month, item, count(date) as date FROM data GROUP BY month, item;")

Elapsed time: 0.007 sec
Elapsed time: 0.040 sec


In [12]:
# Get the sum of the durations per month
data.groupby('month')['duration'].sum() # Produces Pandas Series

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [16]:
data.groupby('month')[['duration']].sum() # Produces Pandas DataFrame

,duration
month,
2014-11,26639.441
2014-12,14641.870
2015-01,18223.299
2015-02,15522.299
2015-03,22750.441


In [44]:
user_usage = pd.read_csv(r'C:\Users\Masha\virtualenvs\tensorflow\data\user_usage.csv', sep=",")
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [45]:
user_device = pd.read_csv(r'C:\Users\Masha\virtualenvs\tensorflow\data\user_device.csv', sep=",")
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [46]:
android_devices = pd.read_csv(r'C:\Users\Masha\virtualenvs\tensorflow\data\android_devices.csv', sep=",")
android_devices.head()

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21
2,NaN,NaN,T31,Panasonic T31
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2
4,3Q,OC1020A,OC1020A,OC1020A


In [47]:
# Запрос на соединение двух наборов данных с помощью pandas
result_merge = pd.merge(user_usage,
                        user_device[['use_id', 'platform', 'device']],
                        on='use_id')
result_merge.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


In [58]:
# Запрос на соединение двух наборов данных с помощью pandasql
print (pysqldf("SELECT outgoing_mins_per_month, monthly_mb, use_id, platform, device FROM user_usage INNER JOIN user_device USING (use_id)").head())

   outgoing_mins_per_month  monthly_mb  use_id platform    device
0                    21.97     1557.33   22787  android  GT-I9505
1                  1710.08     7267.55   22788  android  SM-G930F
2                  1710.08     7267.55   22789  android  SM-G930F
3                    94.46      519.12   22790  android     D2303
4                    71.59     1557.33   22792  android  SM-G361F


In [60]:
# Измерим время выполнения запроса на группировку набора данных с использованием функций агрегирования
# с помощью pandas и pandasql
class Profiler(object):
    def __enter__(self):
        self._startTime = time.time()
         
    def __exit__(self, type, value, traceback):
        print ("Elapsed time: {:.3f} sec".format(time.time() - self._startTime))
        
with Profiler() as p:
    result_merge = pd.merge(user_usage,
                        user_device[['use_id', 'platform', 'device']],
                        on='use_id')
with Profiler() as p:
    pysqldf("SELECT outgoing_mins_per_month, monthly_mb, use_id, platform, device FROM user_usage INNER JOIN user_device USING (use_id)")

Elapsed time: 0.013 sec
Elapsed time: 0.030 sec
